In [ ]:
'''
File 4 of 8 of kdd pipeline

Key features: converting categorical data to numeric
'''

In [ ]:
import pandas as pd

In [62]:
categorical_df = pd.read_csv('../dataset/categorical_data.csv')
numerical_df = pd.read_csv('../dataset/numerical_data.csv')

numerical_df.info()

/var/folders/xk/fhgbj8qj60n54b05mpfnck0w0000gn/T/ipykernel_4408/4095460919.py:1: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  categorical_df = pd.read_csv('../dataset/categorical_data.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155361 entries, 0 to 155360
Data columns (total 34 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   nsites                                        155361 non-null  int64  
 1   nelements                                     155361 non-null  int64  
 2   volume                                        155361 non-null  float64
 3   density                                       155361 non-null  float64
 4   density_atomic                                155361 non-null  float64
 5   deprecated                                    155361 non-null  int64  
 6   energy_per_atom                               155361 non-null  float64
 7   formation_energy_per_atom                     155361 non-null  float64
 8   energy_above_hull                             155361 non-null  float64
 9   is_stable                                     15

In [ ]:
categorical_df.info()


In [49]:
categorical_df['symmetry']

0         {'crystal_system': 'Triclinic', 'symbol': 'P1'...
1         {'crystal_system': 'Triclinic', 'symbol': 'P1'...
2         {'crystal_system': 'Triclinic', 'symbol': 'P1'...
3         {'crystal_system': 'Triclinic', 'symbol': 'P1'...
4         {'crystal_system': 'Triclinic', 'symbol': 'P1'...
                                ...                        
155356    {'crystal_system': 'Triclinic', 'symbol': 'P1'...
155357    {'crystal_system': 'Triclinic', 'symbol': 'P1'...
155358    {'crystal_system': 'Triclinic', 'symbol': 'P1'...
155359    {'crystal_system': 'Triclinic', 'symbol': 'P1'...
155360    {'crystal_system': 'Triclinic', 'symbol': 'P1'...
Name: symmetry, Length: 155361, dtype: object

In [50]:
#begin work on symmetry attribute - these attributes can be cleaned with 1 hot encoding
import ast

crystal_sytems_df = []

for row in categorical_df.itertuples(): #extract eah crystal system from the nested dictionaries for 1 hot encoding
   
    symmetry_dict = ast.literal_eval(row.symmetry) 
    crystal_sytems_df.append(symmetry_dict['crystal_system'])

crystal_sytems_df = pd.get_dummies(
    crystal_sytems_df, columns=['crystal_system'], 
                                    drop_first=True, prefix='crystal_sys')

crystal_sytems_df = crystal_sytems_df.astype(int)
categorical_df = categorical_df.drop('symmetry', axis=1)

numerical_df = pd.concat([numerical_df, crystal_sytems_df], axis=1)


In [63]:
categorical_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155361 entries, 0 to 155360
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   symmetry       155361 non-null  object
 1   structure      155361 non-null  object
 2   decomposes_to  118418 non-null  object
 3   xas            60920 non-null   object
 4   is_gap_direct  155353 non-null  object
 5   is_metal       155353 non-null  object
 6   bandstructure  71727 non-null   object
 7   dos            89547 non-null   object
 8   ordering       154379 non-null  object
 9   bulk_modulus   12259 non-null   object
 10  shear_modulus  12259 non-null   object
dtypes: object(11)
memory usage: 13.0+ MB


In [61]:
#investiage is_metal
print("false count", categorical_df[categorical_df['is_metal'] == 0].shape[0])
print("true count", categorical_df[categorical_df['is_metal'] == 1].shape[0])


false count 81347
true count 74006


In [37]:
#less than 10 are null, fill with 0 (false)
categorical_df['is_metal'] = categorical_df['is_metal'].fillna(0).astype(int) 
categorical_df['is_gap_direct'] = categorical_df['is_gap_direct'].fillna(0).astype(int)

numerical_df = pd.concat([numerical_df, categorical_df[['is_metal', 'is_gap_direct']]], axis=1)

categorical_df = categorical_df.drop(columns=['is_metal', 'is_gap_direct'])

categorical_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155361 entries, 0 to 155360
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   structure      155361 non-null  object
 1   decomposes_to  118418 non-null  object
 2   xas            60920 non-null   object
 3   bandstructure  71727 non-null   object
 4   dos            89547 non-null   object
 5   ordering       154379 non-null  object
 6   bulk_modulus   12259 non-null   object
 7   shear_modulus  12259 non-null   object
dtypes: object(8)
memory usage: 9.5+ MB


In [38]:
categorical_df.drop(['bulk_modulus', 'shear_modulus'], axis=1, inplace=True)
categorical_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155361 entries, 0 to 155360
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   structure      155361 non-null  object
 1   decomposes_to  118418 non-null  object
 2   xas            60920 non-null   object
 3   bandstructure  71727 non-null   object
 4   dos            89547 non-null   object
 5   ordering       154379 non-null  object
dtypes: object(6)
memory usage: 7.1+ MB


In [39]:
categorical_df.head()




,structure,decomposes_to,xas,bandstructure,dos,ordering
0,"{'@module': 'pymatgen.core.structure', '@class...","[{'material_id': 'mp-149', 'formula': 'Si2', '...","[{'edge': 'K', 'absorbing_element': 'Si', 'spe...",{'setyawan_curtarolo': {'task_id': 'mp-1049050...,"{'total': {'1': {'task_id': 'mp-1049056', 'ban...",NM
1,"{'@module': 'pymatgen.core.structure', '@class...","[{'material_id': 'mp-72', 'formula': 'Ti3', 'a...",NaN,NaN,NaN,NM
2,"{'@module': 'pymatgen.core.structure', '@class...","[{'material_id': 'mp-2516584', 'formula': 'C4'...",NaN,NaN,NaN,FM
3,"{'@module': 'pymatgen.core.structure', '@class...","[{'material_id': 'mp-79', 'formula': 'Zn2', 'a...",NaN,NaN,NaN,NM
4,"{'@module': 'pymatgen.core.structure', '@class...","[{'material_id': 'mp-149', 'formula': 'Si2', '...",NaN,NaN,NaN,NM


In [ ]:
numerical_df.to_csv('../dataset/cleaned_numerical_data.csv', index=False)
categorical_df.to_csv('../dataset/cleaned_categorical_data.csv', index=False)